# Assignment 1
## Q3 Cyclone: cycloneanalyzer.ipynb

> __*Gurbaaz Singh Nandra (190349)*__

## Necessary Imports

In [1]:
import math
from typing import Tuple, List
import numpy as np
from PIL import Image

## Reading Images

We can use pillow to open and read an image

In [2]:
Image.open("1.jpg").show()

Ultimately, we are interested in reading all three images into some form of matrices to analyse them. So let us quickly store all three images in `imgs` list

In [3]:
imgs = [np.array(Image.open(f"{i+1}.jpg")) for i in range(3)]

Let us see shape of all three images, and verify if they are same, as for unequal sized images, we would need to bring them down to same dimensions for easier analysis.

In [4]:
for img in imgs:
    print(img.shape)

(626, 940, 3)
(626, 940, 3)
(626, 940, 3)


Indeed, all three images have same dimensions.

## Analysis

Firstly, we would need to identify the location of cyclone cores, which are identified as red in the images. Similarly, we would need to find location of cities and islands, identified as green and blue respectively. Hence the best way to go is to construct a function which identifies if the pixel is red, blue or green

In [5]:
def is_color(pixel: Tuple, color: str) -> bool:
    r, g, b = pixel

    if color == "R":
        # return (r != 0) and (g == 0) and (b == 0)
        return (r > 240) and (g < 50) and (b < 50)

    if color == "G":
        return (g > 240) and (r < 50) and (b < 50)

    if color == "B":
        return (b > 230) and (r < 50) and (g < 50)

Note that we are not equating the `rgb` of a pixel to extremes of the color, e.g., for red (255, 0, 0) since that would be a very strict filter and some color pixel may not actually satsfy that criteria, hence we are doing a loose criteria of checking if the dominant color channel > 240 and rest are < 50.

That being said, now we need to iterate over entire image one by one and collect those points.

In [6]:
def find_color_dots(A: np.ndarray, color: str) -> List[Tuple]:
    assert color == "R" or color == "G" or color == "B"

    color_dots = []

    m, n, _ = A.shape

    for i in range(m):
        for j in range(n):
            pixel = A[(i, j)]

            if is_color(pixel, color):
                color_dots.append((i, j))

    return color_dots

For red dots, the process would look like

In [7]:
red_dots = []

for img in imgs:
    red_dots.append(find_color_dots(img, "R"))
    
red_dots

[[(113, 237),
  (113, 238),
  (113, 239),
  (113, 240),
  (114, 237),
  (114, 238),
  (114, 239),
  (114, 240),
  (179, 547),
  (180, 548),
  (181, 545),
  (181, 548),
  (182, 546),
  (182, 547)],
 [(121, 223),
  (121, 224),
  (122, 223),
  (122, 224),
  (187, 539),
  (187, 540),
  (188, 539),
  (188, 540),
  (189, 539),
  (189, 540),
  (190, 539),
  (190, 540)],
 [(133, 215),
  (133, 216),
  (133, 217),
  (133, 218),
  (134, 215),
  (134, 216),
  (134, 217),
  (134, 218),
  (195, 527),
  (195, 528),
  (195, 529),
  (195, 530),
  (196, 527),
  (196, 528),
  (196, 529),
  (196, 530)]]

Wait! We were hoping to find only two red dots per image, but we are getting way many points in return. This is because even a point on image does not necessarily constitute to a pixel, but actually it contains a collection of pixels, forming a tiny patch on the image. But we need to associate only one pixel coordinate for a dot for purpose of easiness. One way to go about that would be to find average of the coordinate of pixels forming a patch. 

In [8]:
def average_sum_list(A: List[Tuple]) -> List:
    result = []

    for j in range(len(A[0])):

        temp = 0

        for i in range(len(A)):
            temp += A[i][j]

        result.append(temp // len(A))

    return result


def average_dots(A: List[Tuple]) -> List[Tuple]:
    threshold = 10

    if len(A) == 0:
        return []

    avg_dots = []

    temp_dots = []
    temp_dots.append(A[0])

    i = 1

    while i < len(A):

        x, y = temp_dots[0]

        if abs(x - A[i][0]) < threshold and abs(y - A[i][1]) < threshold:
            temp_dots.append(A[i])

        else:
            avg_dots.append(average_sum_list(temp_dots))
            temp_dots = []

            if i != len(A) - 1:
                temp_dots.append(A[i + 1])
                i += 1

        i += 1

    if len(temp_dots) != 0:
        avg_dots.append(average_sum_list(temp_dots))

    return avg_dots

Here we have kept a `threshold` of 10 pixel distance to find neighbouring points.

Let us now find our new `red_dots`.

In [9]:
red_dots = []

for img in imgs:
    red_dots.append(average_dots(find_color_dots(img, "R")))

red_dots

[[[113, 238], [181, 546]], [[121, 223], [188, 539]], [[133, 216], [195, 528]]]

Good, now we have the required points for cyclone cores. We can now find positions of cities in green and islands in blue too.

In [15]:
green_dots = average_dots(find_color_dots(imgs[0], "G"))
blue_dots = average_dots(find_color_dots(imgs[0], "B"))

green_dots, blue_dots

([[267, 200], [436, 139], [617, 94]], [[475, 415], [498, 362]])

It would be cool if we are able to visualise all three positions of each cyclone core over time to see its trajectory. 

In [16]:
ni = np.copy(imgs[0]) # ni: new image

In [17]:
def draw_circle(A: np.ndarray, center: Tuple, radius: int, color: Tuple) -> np.ndarray:
    x0, y0 = center

    for i in range(x0 - radius, x0 + radius):
        for j in range(y0 - radius, y0 + radius):
            if (i - x0) ** 2 + (j - y0) ** 2 <= radius**2:
                A[(i, j)] = color

    return A

In [18]:
for i in red_dots:
    for j in i:
        ni = draw_circle(ni, tuple(j), 3, (255, 0, 0))

Image.fromarray(ni, 'RGB').show()

We can see that cores of both of the cyclones are moving in a non-linear path, but from the problem statement, we know that as cyclones approach land, their inertia increases and they continue along the same route. Hence we can make this assumption and assume that from here cyclones would be moving in a straight path.

For this, we need to only consider the trajectory as extrapolated from last two points of the image. All we need to do is find equations of line for trajectory of both cyclones and find their point of intersection, which is the apojuncture. To find if they would actually collide, we would need to check if their times of reaching that apojuncture point (`time = velocity / distance`) is same. 

In [19]:
def distance(a: Tuple, b: Tuple) -> float:
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


def velocity(a: Tuple, b: Tuple, time: int) -> float:
    return distance(a, b) / time

In [20]:
def find_line(c: Tuple, p: Tuple) -> Tuple:
    m = (c[1] - p[1]) / (c[0] - p[0])
    c = p[1] - m * p[0]
    return (m, c)


def satisfy_line(theta: Tuple, p: Tuple) -> bool:
    return abs((theta[1] + theta[0] * p[0]) - p[1]) <= 1

In [21]:
m1, c1, m2, c2 = *find_line(red_dots[2][0], red_dots[1][0]), *find_line(red_dots[2][1], red_dots[1][1])

In [22]:
point_of_intersection = int((c2 - c1) / (m1 - m2)), int((m1 * c2 - m2 * c1) / (m1 - m2))

point_of_intersection

(547, -25)

We can see that the point of intersection is outside the image coordinates, which is perfectly possible.

Let us plot the trajectories on the image and visualise it

In [23]:
for m_c in [(m1, c1), (m2, c2)]:
    for i in range(ni.shape[0]):
        for j in range(ni.shape[1]):
            if satisfy_line(m_c, (i, j)):
                ni[(i, j)] = (165, 42, 42)

In [24]:
Image.fromarray(ni, 'RGB').show()

To answer the first part of the question `How far (in Km) the apojuncture will be from the capital city?`, we need to translate the pixel distance to kilometeres. 

We have been provided that `The islands of Reunion and Mauritius (blue) areseparated by 250 kilometres to its east` and `Madagascar is located on an island in the Indian Ocean. It stretches 1500 kilometres from
north Antsiranana (green) to south Benonoka (green)`. Using both these information, let us find km to pixel ratio.

In [25]:
(distance(blue_dots[0], blue_dots[1]) / 250), (distance(green_dots[0], green_dots[1]) / 1500) 

(0.23110170921046863, 0.11978128215858919)

Surpirsingly, both these ratios are not same! This is because the satellite image is not a flat image, but rather a distorted one. The main point of capture would have been the middle point of line joining the cyclone cores (call it `X`). To simply the distortion, we can use the pixel to km ratio obtained from green dots for the part of image on left of X, and the ratio obtained from blue dots for the right part. Thus, to find actual distance of apojuncture from Ambali Cyclone, we would need to use both of the ratios for the relavant part of its trajectory. Wheras, for Cyclone Belna, the ratio obtained from green dots would solely be used to find actual distance from apojuncture.

In [26]:
def mid_point(a: Tuple, b: Tuple) -> Tuple:
    return ((a[0] + b[0]) // 2, (a[1] + b[1]) // 2)

In [27]:
X = mid_point(red_dots[2][0], red_dots[2][1])

X

(164, 372)

Equation of such a line would simply be `x = X[1]`

In [28]:
for i in range(ni.shape[0]):
    for j in range(ni.shape[1]):
        if j == X[1]:
            ni[(i, j)] = (255, 255, 255)

In [29]:
Image.fromarray(ni, 'RGB').show()

Now we are in a position to find the distances of cyclone cores from apojuncture in kilometers.

Let the point of intersection of vertical line dividing image in two parts and trajectory of cyclone Ambali be `Y`

In [30]:
r1, r2 = (250 / distance(blue_dots[0], blue_dots[1])), (1500 / distance(green_dots[0], green_dots[1])) 

In [31]:
dist_belna = distance(point_of_intersection, red_dots[2][0]) * r2

dist_belna

3999.269069268583

In [32]:
Y = (X[1], int(m2 * X[1] + c2))

dist_ambali = distance(point_of_intersection, Y) * r2 + distance(Y, red_dots[2][1]) * r1

dist_ambali

4143.966436752325

#### Q1. How far (in Km) the apojuncture will be from the capital city?

In [33]:
## Let the answer be ans_1

ans_1 = distance(point_of_intersection, green_dots[2]) * r2

str(ans_1) + " km"

'1152.6139808757434 km'

#### Q2.  When will it occur (how many hours after the last satellite image capture)?

In [34]:
## Let the answer be ans_2

## We should also note the timestamps between image here, to compute the speed of cyclone cores, which would be 
## assumed constant from here

velocity_belna = velocity(red_dots[1][0], red_dots[2][0], 55) * r2 ## 55 = 15:39 - 14:44 
velocity_ambali = velocity(red_dots[1][1], red_dots[2][1], 55) * r1

velocity_belna, velocity_ambali

(2.1087592832980593, 1.0257903606755068)

In [35]:
## As time of both the cyclone cores reaching the apojuncture could be different, we need to calculate time
## taken by both cyclones

time_belna = dist_belna / velocity_belna
time_ambali = dist_ambali / velocity_ambali

time_belna, time_ambali

(1896.5033614523334, 4039.779077299407)

#### Q3. Will they actually collide in reality?

Nope, since even if we make these heavy assumptions of cyclone cores moving in straight path with constant speed right after 15:39 hours, they would meet at apojuncture times owing to their different time taken (time = speed / distance ratio). Furthermore, the actual trajectories of these cyclones are far more complicated, and they did not even collide in real life, as cyclone ambali died out gradually way before meeting cyclone belna.